In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import norm

Die Daten pro Station gibt es hier: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/annual/kl/historical/. Einfach die txt-Datei speichern und den Ort als Parameter der draw_chart-Funktion übergeben.

Die Idee der Darstellung stammt von [Ed Hawkins](https://twitter.com/ed_hawkins?lang=de), der [#showyourstripes](https://showyourstripes.info/) ins Leben gerufen hat. 

In [2]:
def wrangle_data(url):
    #Daten einlesen
    df = pd.read_csv(url, sep=";", dtype=object)
    
    # Jahreszahl festlegen
    df["jahr"] = df["MESS_DATUM_ENDE"].apply(lambda x: x[:4])
    
    # Spalten löschen und umbenennen
    cols = ['STATIONS_ID', 'jahr', 'JA_TT']
    df = df[cols]
    df.columns = ["id", "jahr", "temp"]
    
    # richtigen Datentyp festlegen
    df["temp"] = df["temp"].astype(float)
    df["jahr"] = pd.to_datetime(df["jahr"], format="%Y")
    
    # Einen festen Wert festlegen, damit am Ende alle Balken die gleiche Höhe haben
    df["value"] = 10
    
    # Fehlende Werte aussortieren
    df = df.replace(-999.0, np.nan)
    df = df.dropna()
    
    # Die Temperaturschwanken zwischen 0 und 1 normalisieren
    df["temp_normalize"] = round((df.temp - df.temp.min()) / (df.temp.max() - df.temp.min()),2)
    
    return df


In [30]:
def draw_chart(url, name):
    
    # Die Daten laden und bearbeiten
    df = wrangle_data(url)
    
    # Den Farbverlauf festlegen 
    cmap = plt.get_cmap("seismic")

    # Die Variablen festlegen, die geplottet werden sollen
    height = df.value.values
    bars = df.jahr.values
    color = df.temp_normalize.values
    y_pos = np.arange(len(bars))
    
    # Größe der Grafik festlegen
    mydpi = 100
    fig, ax = plt.subplots(figsize = [644 * 2 / mydpi, 400 * 2 / mydpi])

    # Balken zeichen
    plt.bar(y_pos, height, color=cmap(color))

    # Alle Achsen und Beschriftungen entfernen
    sns.despine(left=True, bottom=True)
    plt.tick_params(
        axis='both',
        which='both',
        bottom=False,
        top=False,
        left=False,
        labelbottom=False,
        labelleft=False)
    plt.tight_layout()

    # Als png speichern
    plt.savefig("./vis/"+name+".png", transparent=True, dpi=100)
    
    min_jahr = str(df.jahr.min())[:4]
    max_jahr = str(df.jahr.max())[:4]
    min_temp = df.temp.min()
    max_temp = df.temp.max()
    
    print("Von {} bis {}.".format(min_jahr, max_jahr))
    print("Niedrigste Temp: {}, höchste Temp: {}".format(min_temp, max_temp))
    
    print("Werte von {} bis {}. Niedrigste Temperatur: {} Grad Celsius (tiefes blau), höchste Temperatur: {} Grad Celsius (tiefes rot).".format(min_jahr, max_jahr, min_temp, max_temp) )
    print(df[df.temp == min_temp])
    
    